Heigon Soldera 217638  
Rafael Galib 204904

# Introdução

Para esta atividade nos foi fornecida a ferramenta disponível em http://cs.brynmawr.edu/Courses/cs372/fall2017/Code/search.zip.
Com ela podemos simular o jogo do Pac-Man, sendo que nos é disponível uma coleção de funcionalidades já implementadas, assim como uma coleção de layouts para as simulações.


No pacote de arquivos, temos o motor do jogo, que conta com alguns arquivos-chave que merecem ser comentados:
* pacman.py;
* game.py;

Ambos os arquivos utilizam de outros inerentes do pacote que não serão abordados, mas auxiliam no funcionamento do motor.
O arquivo pacman.py é o arquivo a ser rodado por meio do comando !python pacman.py. Devido a estrutura da implementação deste arquivo, podemos passar parâmetros no comando para efetuar as simulações que precisamos.


Os principais parâmetros são:

* --layout {layout}: com este comando fazemos a seleção de qual layout específico desejamos simular;
* --pacman {agent}: com este comando escolhemos o agente. O agente é a classe que contém a lógica de nosso pacman;
* --numGames {nGames}: com este comando escolhemos quantas vezes queremos que um determinado agente jogue um determinado layout. O próprio motor do jogo computa as médias das pontuações, assim como o histórico de vitórias e derrotas;
* --quetTextGraphics: este comando faz com que a parte gráfica da simulação não seja exibida na tela caso um agente tenha sido utilizado de argumento. Isso poupa muito processamento e acelera a simulação;


    Exemplo de utilização:
              !python pacman.py --layout smallClassic --pacman GeneticAgent --numGames 5 --quietTextGraphics

Para a implementação da lógica a ser feita pelo Pac-Man, precisamos criar um arquivo terminado em 'gents.py', no caso, criamos o Agents.py. O pacote já vem com alguns arquivos 'gents.py' contendo exemplos de agentes já implementados.
No arquivo Agents.py encontra-se a classe de agente que foi utilizada no exemplo de comando logo acima. Nesta classe encontra-se a estrutura necessária, seguindo o padrão de outros agentes de exemplo, para que o motor possa simular a nossa lógica.

# O Problema

Uma vez entendido e preparado o motor do Pac-Man iremos utilizar os conceitos recém aprendidos na disciplina para abordar de duas maneiras distintas as simulações do Pac-Man: através de um algoritmo genético e de um algoritmo de aprendizado por reforço. Para esta atividade escolhemos o Q-Learning como algoritmo de aprendizado por reforço.

Pede-se que os algoritmos treinem em 3 diferentes layouts: 
* smallClassic;
* mediumClassic;
* originalClassic;

Pede-se também que se colha e compare os resultados dos dois algoritmos.

# Evolutionary Model

A seguir serão discutidos alguns pontos-chave da implementação do modelo, assim como a teoria por trás das decisões de concepções adotadas no projeto. Para a apresentação e discussão dos resultados, a parte executável do código já foi processada e sua saída será utilizada para as conclusões. O motivo disso é o custo computacional inerente do modelo evolutivo, que torna inviável que se aguarde a execução a fim de apreciar os resultados.

# O modelo em si

A concepção do modelo evolutivo dá-se pela concepção de que: sob um determinado ambiente, podemos fazer uma seleção de indivíduos sob um certo viés, o que resulta na disseminação de determinadas características à futuras gerações destes indivíduos.

Para tanto, o ambiente deve seguir determinados critérios e a seleção deve refletir o ambiente. Assim, determinamos certos parâmetros para controlar o pacman da seguinte forma:

* Primeiro parâmetro indica distância do fantasma mais próximo a qual o pacman deve fugir:
    * Se o pacman está fugindo:
        * Segundo parâmetro indica uma ordem de moiventos a se seguir para fugir dos fantasma (como consideramos 8 posições relativas para o fantasma teremos 8 parâmetros)
    * Se o pacman está perseguindo comida:
        * Terceiro parâmetro indica uma probabilidade para cada movimento legal (como consideramos 8 posições relativas para a comida teremos 8 parâmetros)
* Um parâmetro adicional que indica quantos movimentos o pacman pode não se preocupar com fantasmas após ter comido uma cápsula.

A seguir será descrito as técnicas utilizadas para o desenvolvlimento e otimização desse modelo com esses parâmetros 

## Population size

Primeiramente, é necessária uma população de indivíduos que irão interagir com o ambiente. A população para o layout smallClassic será 200 indivíduos, mediumClassic 100 e originalClassic 50 para poupar poder computacional.

## Fitness function

A estes indivíduos, será utilizada uma função, tradicionalmente chamada de $fitness$, que, a partir de determinados resultados obtidos da interação dos indivíduos com o ambiente, atribuirá uma pontuação ao indivíduos, o que nos permite classificar os indivíduos sob nosso critério de seleção, ela é da seguinte forma:  
  
  * $fitness(indivíduo) = vitórias + comidas \times 2 + cápsulas + min(1, tempo_de_execução) $  
  
## Replacement 

Feita a classificação, propomos que uma nova população será gerada a partir da combinação das características dos indivíduos. Tradicionalmente gera-se uma população nova com a mesma quantidade de indivíduos que a anterior, denominando-se assim como uma nova geração. Esta nova geração será composta de indivíduos que refletirão características de seus antecessores, sendo então necessária uma concepção de hereditariedade.

## Selection

Para que uma nova geração reflita seus antecessores, fazemos tantas seleções de indivíduos antepassados quanto forem necessárias e dessas seleções fazemos a combinação das características, e dessa combinação resulta um novo indivíduo. Em geral, combina-se dois indivíduos antepassados para gerar um novo indivíduo. A seleção de antepassados pode ser realizada de várias formas, a nível de exemplo: neste projeto selecionamos os 5 indivíduo mais bem adaptados e selecionamos 2 ao acaso, sendo que a chance de um indivíduo ser selecionado é proporcional à pontuação de sua função fitness. Repetindo essa seleção de pares quantas vezes forem necessárias, formamos uma nova geração de indivíduos. Metódo conhecido como Roulette Wheel.

## CrossOver

As características dos indivíduos a serem combinadas em geral são valores numérios cuja finalidade varia de implementação para implementação. No nosso modelo alguns valores simbolizam distâncias a serem consideradas, enquanto que outros valores servem como coeficientes para ações a serem tomadas em diversas situações possíveis. A esse conjunto de valores damos o nome de DNA, ou código genético, enquanto que os valores em si são denomindados alelos. A combinação dessas características também é bastante flexível. No nosso código, para cada alelo temos 50% de chance de herdar a característica de algum dos indivíduos selecionados no par. Esse método é chamado de Uniform CrossOver.

## Mutation

Prevendo que características menos eficazes podem disseminar-se contundentemente pelas gerações futuras, incluímos, por fim, uma chance de mutação em algum indivíduo, sendo que essa mutação consiste na alteração de algum alelo para algum valor aleatório que respeite os limites do alelo em questão. No nosso código cada novo indivíduo tem 15% de chance de passar pelo processo de mutação implementado por nós, que, para cada alelo presente no código genético do novo indivíduo, possui 10% de chance de trocar seu valor. Esse método é chamado de Uniform Mutation.

## Stop Criteria

O métdo para de executar somente quando encontrar um indivíduo que ganha uma quantidade mínima de jogos que varia para cada layout.

# Implementação do Indivíduo

O código dos indivíduos está escrito no arquivo Agents.py, onde lá encontra-se a classe GeneticAgent, que é o agente que possui o comportamento implementado por nós.

No código temos métodos de apoio assim como métodos evocados pelo próprio motor da ferramenta fornecida a nós.

O seguinte trecho de código é executado a cada iteração do jogo. Nele constitui-se o comportamento do agente.
Primeiramente, utilizamos os métodos de apoio para calcular algumas informações como: distância ao fantasma mais próximo, distância a comida mais próxima, assim como suas direções relativas. Uma vez que exista um fantasma próximo o suficiente do pacman, o mesmo dará preferência por evitar os fantasmas, enquanto que caso não haja um fantasma suficientemente próximo, o pacman irá dar preferência a comidas.
No código, quebramos todas as possíveis combinações de entornos que o pacman encontraria a sua volta, e, para estes, o pacman possui diferentes preferências para direções a seguir. Estas preferências são passadas por parâmetro ao pacman, constituindo seu código genético.

        def getAction(self, state):
                "The agent receives a GameState (defined in pacman.py)."
                self.moves += 1
                rw = ReadWrite()
                posG = np.array(state.getGhostPositions())
                posP = np.array(state.getPacmanPosition())
                posCaps = np.array(state.getCapsules())
                posFood = state.getFood()
                
                if(self.numCaps == None):
                    self.numCaps = len(posCaps)
                elif(self.numCaps != len(posCaps)):
                    self.numCaps = len(posCaps)
                    self.Attack = 0
                elif(self.Attack >= rw.pmt[1]):
                    self.Attack = None

                if(self.Attack != None):
                    self.Attack += 1

                minDistGhost = 99999
                indG = None
                for i in range(len(posG)):
                    if(np.linalg.norm(posG[i]-posP) < minDistGhost):
                        minDistGhost = np.linalg.norm(posG[i]-posP)
                        indG =  i

                if(posG[indG][0] > posP[0]):
                    if(posG[indG][1] > posP[1]):
                        ghostRelativePosition = "upright"
                    elif(posG[indG][1] == posP[1]):
                        ghostRelativePosition = "right"
                    else:
                        ghostRelativePosition = "downright"

                elif(posG[indG][0] == posP[0]):
                    if(posG[indG][1] > posP[1]):
                        ghostRelativePosition = "up"
                    else:
                        ghostRelativePosition = "down"
                else:
                    if(posG[indG][1] > posP[1]):
                        ghostRelativePosition = "upleft"
                    elif(posG[indG][1] == posP[1]):
                        ghostRelativePosition = "left"
                    else:
                        ghostRelativePosition = "downleft"

                minDistFood = 99999
                minFood = None
                for x in range(posFood.width):
                    for y in range(posFood.height):
                        if(posFood[x][y]):
                            food = np.array([x,y])
                            distFood = np.linalg.norm(food-posP)
                            if(distFood<minDistFood):
                                minDistFood = distFood
                                minFood = food
                
                if(minFood[0] > posP[0]):
                    if(minFood[1] > posP[1]):
                        foodRelativePosition = "upright"
                    elif(minFood[1] == posP[1]):
                        foodRelativePosition = "right"
                    else:
                        foodRelativePosition = "downright"

                elif(minFood[0] == posP[0]):
                    if(minFood[1] > posP[1]):
                        foodRelativePosition = "up"
                    else:
                        foodRelativePosition = "down"
                else:
                    if(minFood[1] > posP[1]):
                        foodRelativePosition = "upleft"
                    elif(minFood[1] == posP[1]):
                        foodRelativePosition = "left"
                    else:
                        foodRelativePosition = "downleft"

                if(len(state.getLegalPacmanActions()) == 2 ):
                    path = "deadend"
                else:
                    path = None

                if(path == "deadend"):
                    if (Directions.SOUTH in state.getLegalPacmanActions()):
                        return Directions.SOUTH
                    elif (Directions.NORTH in state.getLegalPacmanActions()):
                        return Directions.NORTH
                    elif (Directions.WEST in state.getLegalPacmanActions()):
                        return Directions.WEST
                    else:
                        return Directions.EAST

                elif(minDistGhost < rw.pmt[0] and self.Attack == None):
                    if(ghostRelativePosition == "right"):
                        return self.directions(state,self.PossibleDirections[rw.pmt[2]])

                    elif(ghostRelativePosition == "left"):
                        return self.directions(state,self.PossibleDirections[rw.pmt[3]])

                    elif(ghostRelativePosition == "up"):
                        return self.directions(state,self.PossibleDirections[rw.pmt[4]])

                    elif(ghostRelativePosition == "down"):
                        return self.directions(state,self.PossibleDirections[rw.pmt[5]])

                    elif(ghostRelativePosition == "downright"):
                        return self.directions(state,self.PossibleDirections[rw.pmt[6]])

                    elif(ghostRelativePosition == "downleft"):
                        return self.directions(state,self.PossibleDirections[rw.pmt[7]])
                    
                    elif(ghostRelativePosition == "upleft"):
                        return self.directions(state,self.PossibleDirections[rw.pmt[8]])
                    
                    elif(ghostRelativePosition == "upright"):
                        return self.directions(state,self.PossibleDirections[rw.pmt[9]])
                else:
                    if(foodRelativePosition == "right"):
                        return self.directionsProb(state=state,p1=rw.pmt[10],p2=rw.pmt[11],p3=rw.pmt[12],p4=rw.pmt[13])

                    elif(foodRelativePosition == "left"):
                        return self.directionsProb(state=state,p1=rw.pmt[14],p2=rw.pmt[15],p3=rw.pmt[16],p4=rw.pmt[17])

                    elif(foodRelativePosition == "up"):
                        return self.directionsProb(state=state,p1=rw.pmt[18],p2=rw.pmt[19],p3=rw.pmt[20],p4=rw.pmt[21])

                    elif(foodRelativePosition == "down"):
                        return self.directionsProb(state=state,p1=rw.pmt[22],p2=rw.pmt[23],p3=rw.pmt[24],p4=rw.pmt[25])

                    elif(foodRelativePosition == "downright"):
                        return self.directionsProb(state=state,p1=rw.pmt[26],p2=rw.pmt[27],p3=rw.pmt[28],p4=rw.pmt[29])

                    elif(foodRelativePosition == "downleft"):
                        return self.directionsProb(state=state,p1=rw.pmt[30],p2=rw.pmt[31],p3=rw.pmt[32],p4=rw.pmt[33])
                    
                    elif(foodRelativePosition == "upleft"):
                        return self.directionsProb(state=state,p1=rw.pmt[34],p2=rw.pmt[35],p3=rw.pmt[36],p4=rw.pmt[37])
                    
                    elif(foodRelativePosition == "upright"):
                        return self.directionsProb(state=state,p1=rw.pmt[38],p2=rw.pmt[39],p3=rw.pmt[40],p4=rw.pmt[41])


Outro método importante está no arquivo ReadWrite.py, mais especificamente o construtor da classe. No construtor lemos os parâmetros que compõem o código genético do indivíduo. Perceba a manipulação de diversos arquivos .txt que foram utilizado para a comunicação entre diferentes módulos da ferramenta.
        def __init__(self):

            with open('nome.txt', 'r') as f:
                lines = f.readlines()
                self.nome = lines[0]

            with open('agentNum_' + self.nome + '.txt') as f:
                lines = f.readlines()
                self.agentNum = int(lines[0])

            with open('parameters_' + self.nome + '.txt') as f:
                lines = f.readlines()
                
                self.pmt = lines[self.agentNum].split()

                for i in range(len(self.pmt)):
                    if(1<=i<=9):
                        self.pmt[i] = int(self.pmt[i])
                    else:
                        self.pmt[i] = float(self.pmt[i])

Com isso chegamos ao ultimo método aqui listado, pertencente à classe GeneticAgent. Temos aqui o metodo final(), que é evocado pela ferramenta no final de cada jogo, e com isso escrevendo resultados relativos a cada indivíduo da população.

        def final(self, state):
            posCaps = np.array(state.getCapsules())

            with open('moves.txt', 'w') as f:
                f.write(str(self.moves) + '\n')

            print ("Acabou",posCaps)
    


# Implementação do modelo

Para a implementação do modelo e seu funcionamento, utilizamos diversos métodos de apoio localizados mais adiante neste notebook.

A começar pelas dependências de importações: 

        from numpy.random import choice
        import numpy as np
        import random
        import matplotlib.pyplot as plt

A seguir temos um dos métodos mais importantes, o método crossOver(). Nele fazemos a combinação de dois indivíduos (p1 e p2), gerando por fim um novo indivíduo (child).

        def crossOver(p1,p2):
                dadOrmom = [False, True]
                child = []

                cross = choice(dadOrmom, p=[0.5,0.5])

                if(cross):
                        child.append(p1[0])
                else:
                        child.append(p2[0])
                
                for i in range(1,10):
                        cross = choice(dadOrmom, p=[0.5,0.5])
                        if(cross):
                        child.append(p1[i])
                        else:
                        child.append(p2[i])

                for i in range(10,18):
                        cross = choice(dadOrmom, p=[0.5,0.5])

                        if(cross):
                
                                index = (i-9)*4+9
                                child.append(p1[index-3])
                                child.append(p1[index-2])
                                child.append(p1[index-1])
                                child.append(p1[index-0])
                        else:
                        
                                index = (i-9)*4+9
                                child.append(p2[index-3])
                                child.append(p2[index-2])
                                child.append(p2[index-1])
                                child.append(p2[index-0])
                return child

Aqui está um de nosso métodos de apoio, que gera 4 numeros aleatórios cuja soma resulta em 1

                def random4():
                        p1 = np.random.uniform(low=0.0, high=1)
                        p2 = np.random.uniform(low=0.0, high=(1-p1))
                        p3 = np.random.uniform(low=0.0, high=(1-p1-p2))
                        p4 = 1-(p1+p2+p3)
                        while(p1+p2+p3+p4!=1):
                                p1 = np.random.uniform(low=0.0, high=1)
                                p2 = np.random.uniform(low=0.0, high=(1-p1))
                                p3 = np.random.uniform(low=0.0, high=(1-p1-p2))
                                p4 = 1-(p1+p2+p3)
                        return p1,p2,p3,p4

Em sequência temos o método responsável pela mutação do novo indivíduo (child). Nele, cada alelo possui 10% de chance de ser alterado respeitando os valores limites que ele pode assumir.

                def mutationChild(child):
                        mutation = [False, True]

                        ifMut = choice(mutation, p=[0.9,0.1])
                        if(ifMut):
                                child[0] = np.random.uniform(low=0.0, high=5.0)
                        
                        for i in range(1,10):
                                ifMut = choice(mutation, p=[0.9,0.1)
                                if(ifMut):
                                if(i == 1):
                                        child[i] = random.randint(0, 30)
                                else:
                                        child[i] = random.randint(0, 23)
                        
                        for i in range(10,18):
                                ifMut = choice(mutation, p=[0.9,0.1)
                                if(ifMut):
                                
                                index = (i-9)*4+9
                                p1,p2,p3,p4 = random4()
                                child[index-3] = p1
                                child[index-2] = p2
                                child[index-1] = p3
                                child[index-0] = p4
                        return child      

Temos então mais um método de apoio que nos auxilia na conversão de listas em strings.

                def writeString(child):
                        string = ""
                        for i in range(len(child)):
                                string += str(child[i])+" "
                        string+="\n"
                        return string

Por fim temos o método geraPopIni, que gera combinações aleatórias de códigos genéticos para serem utilizados como geração inicial do experimento.

                def geraPopIni(nome, pop):   
                        import numpy as np
                        import random
                        

                        string = "" + str(np.random.uniform(low=0.0, high=5.0))

                        string += " " + str(random.randint(0, 30))

                        for i in range(8):
                                string += " " + str(random.randint(0, 23))

                        for i in range(8):
                                p1,p2,p3,p4 = random4()
                                string += " " + str(p1) + " " + str(p2) + " " + str(p3) + " " + str(p4)

                        with open('parameters_' + nome + '.txt','w') as f:
                                f.write(string)



                        for i in range(pop - 1):

                                string = "\n" + str(np.random.uniform(low=0.0, high=5.0))

                                string += " " + str(random.randint(0, 30))

                                for i in range(8):
                                string += " " + str(random.randint(0, 23))

                                for i in range(8):
                                p1,p2,p3,p4 = random4()
                                string += " " + str(p1) + " " + str(p2) + " " + str(p3) + " " + str(p4)
                                        
                                with open('parameters_' + nome + '.txt','a') as f:
                                f.write(string)




# Execução do experimento

Arquitetado todo o aparato necessário para os experimentos, podemos então utilizá-lo num único método que será responsável por reproduzir o que foi mencionado na seção do modelo genético:  
* Inicializamos as variáveis necessárias, assim como os arquivos de saída necessários. É gerada também a população inicial do experimento utilizando o método geraPopIni();
* Realizamos em loop uma sequência de etapas:
    * Para cada indivíduo executamos a simulação com o comando !python pacman.py, que aceita parâmetros para sua execução.
    * Feitas as simulações, fazemos a leitura do arquivo de resultados e classificamos os indivíduos a partir de seu fitness.
    * Neste ponto é verificado o critério de parada do experimento. Como a simulação exige um certo custo computacional, além do tempo necessário para simulações muito longas, resolvemos restringir o quanto iremos iterar sobre o problema. Resolvemos então adotar um critério que leva em conta as vitórias de um indivíduo, assim como algumas de suas pontuações. Uma vez que um indivíduo ganha um determinado número de jogos, salvamos suas informações num arquivo específico. É dada então uma margem de poucas gerações para que um indivíduo tenha um desempenho melhor e tenha suas informações armazenadas. Restringimos a no máximo 5 gerações, assim como um limite de 10 trocas de melhor indivíduo. Caso nenhum novo melhor indivíduo seja encontrado, ou estejam esgotadas as trocas, encerramos a simulação.
    * Caso não seja a hora de encerrar, formamos pares dentre os 5 indivíduos com melhor classificação. A escolha do indivíduo para compor o par é proporcional à sua pontuação. Formamos pares o suficientes para gerar toda uma nova população.
    * A cada indivíduo gerado é dada 15% de chance de ele passar pelo processo de mutação através do método mutationChild(), onde cada alelo de seu código genético terá 10% de chance de ser alterado.
    
    

        def treina(layout, nGames, minVic, pop):
            from numpy.random import choice
            import numpy as np
            import time

            start_time = time.time()

            population = pop
            count = 6
            NotStop = True
            hasBest = False

            bCount = 0

            nome = layout
            geraPopIni(nome, pop)

            with open('nome.txt', 'w') as f:
                f.write(nome)

            generation = 0
            with open('generation_' + nome + '.txt','w') as f:
                f.write("")

            with open('fitness_' + nome + '.txt','w') as f:
                f.write("")

            with open('Best_' + nome + '.txt','w') as f:
                f.write("")

            while(NotStop):

                # Run games

                with open('results_' + nome + '.txt','w') as f:
                        f.write("")
                        f.close()
                for i in range(population):

                    with open('agentNum_' + nome + '.txt','w') as f:
                        f.write(str(i))

                    saida = !python pacman.py --layout {layout} --pacman GeneticAgent --numGames {nGames} --quietTextGraphics
                    
                allPmt = []
                allRes = []
                allRes2 = []
                inds = []

                # Read parameters 
                for k in range (population):
                    with open('parameters_' + nome + '.txt') as f:
                        lines = f.readlines()

                        pmt = lines[k].split()

                    for j in range(len(pmt)):
                        if(1<=j<=9):
                            pmt[j] = int(pmt[j])
                        else:
                            pmt[j] = float(pmt[j])
                    
                    allPmt.append(pmt)

                # Read results
                for k in range (population):
                    with open('results_' + nome + '.txt') as f:
                        lines = f.readlines()

                        res = lines[k].split()

                    for j in range(len(res)):
                        res[j] = float(res[j])
                    
                    allRes.append(res)


                # Select the best 5

                fitList = np.array([])

                for k in range (population):
                    fit = allRes[k][1]+((229-allRes[k][3])/229)*2 + ((4-allRes[k][4])/4) + (min(allRes[k][5],1))
                    fitList = np.append(fitList,fit)

                inds = np.argpartition(fitList, -5)[-5:]
                indsWorsts = np.argpartition(fitList, 5)[:5]

                print("Gen: ",generation)
                print("Best 5:")
                for i in inds:
                    print(i,fitList[i],allRes[i])
                print("Worst 5:")
                for i in indsWorsts:   
                    print(i,fitList[i],allRes[i])
                print()

                avgFit = sum(fitList)/len(fitList)
                avgBestFit = 0
                avgWorstFit = 0

                for i in inds:
                    avgBestFit += fitList[i]
                avgBestFit = avgBestFit/5
                for i in indsWorsts:
                    avgWorstFit+= fitList[i]
                avgWorstFit = avgWorstFit/5

                with open('fitness_' + nome + '.txt','a') as f:
                    s = str(avgBestFit) + " " + str(avgFit) + " " + str(avgWorstFit) + "\n"
                    f.write(s)

                for i in inds:
                    if(allRes[i][1]>=minVic):

                        if hasBest:                  

                            if fitList[i] > best[0] and allRes[i][0] > best[1] and allRes[i][1] > best[2]:
                                best = [fitList[i], allRes[i][0], allRes[i][1]]
                                with open('Best_' + nome + '.txt','w') as f:
                                    f.write(writeString(allPmt[i]))
                                    f.write(writeString([fitList[i])])
                                    f.write(writeString(allRes[i]))
                                    spentTime = (time.time() - start_time)
                                    f.write("Execution Time: "+str(spentTime) + '\n')
                                bCount += 1
                                count = 6                  

                        else:
                            hasBest = True

                            best = [fitList[i], allRes[i][0], allRes[i][1]]
                            bCount += 1
                            with open('Best_' + nome + '.txt','w') as f:
                                f.write(writeString(allPmt[i]))
                                f.write(writeString([fitList[i])])
                                f.write(writeString(allRes[i]))
                                spentTime = (time.time() - start_time)
                                f.write("Execution Time: "+str(spentTime) + '\n')
                
                if (hasBest):
                    count -= 1
                    if count == 0 or bCount == 10:
                        NotStop = False 

                if(NotStop):

                    with open('parameters_' + nome + '.txt','w') as f:
                        f.write("")


                    # CrossOver

                    childs = []

                    for k in range(int(population)):               
                        d = choice(inds, p=fitList[inds]/fitList[inds].sum())
                        m = choice(inds, p=fitList[inds]/fitList[inds].sum())
                        while(d==m):
                            m = choice(inds, p=fitList[inds]/fitList[inds].sum())


                        child = crossOver(allPmt[d],allPmt[m])

                        # Mutation
                        if choice([True, False], p=[0.15, 0.85]):
                            child = mutationChild(child)


                        childs.append(child)

                    with open('parameters_' + nome + '.txt','a') as f:
                        for k in range(population):
                            f.write(writeString(childs[k]))

                    

                    with open('generation_' + nome + '.txt','a') as f:
                        f.write(str(generation)+"\n")
                    generation+=1

# Realizando as simulações

Estando tudo pronto, podemos então realizar as simulações nos cenários requisitados pela atividade: smallClassic, mediumClassic e originalClassic.
Cada cenário é simulado com parâmetros distintos:

* Número de jogos a serem realizados por cada indivíduo;
* Número mínimo de vitórias para o critério de parada;
* Número de indivíduos das gerações;


        nGames = {"smallClassic" : 9, "mediumClassic" : 6, "originalClassic" : 3}
        minVic = {"smallClassic" : 6, "mediumClassic" : 4, "originalClassic" : 1}
        pops =   {"smallClassic" : 200, "mediumClassic" : 100, "originalClassic" : 50}

        for layout in ["smallClassic", "mediumClassic", "originalClassic"]:
            treina(layout, nGames[layout], minVic[layout], pops[layout])

# Apresentação dos resultados

Executadas todas as simulações temos como resultado, para cada geração, as médias dos 5 melhores indivíduos e por fim rodamos nosso melhor indivíduo 10 vezes para cada layout, a média geral da população e por fim a média dos 5 piores indivíduos. Com o seguinte trecho de código imprimimos.

        for layout in ["smallClassic", "mediumClassic", "originalClassic"]:
            Best = []
            Avg = []
            Wrst = []
            with open('fitness_' + layout + '.txt', 'r') as f:
                lines = f.readlines()
                for line in lines:
                    line = line.split()
                    Best.append(float(line[0]))
                    Avg.append(float(line[1]))
                    Wrst.append(float(line[2]))
                
                x = np.arange(len(lines))
            
            
            plt.plot(x, Best, label = 'Best') 
            plt.plot(x, Avg, label = 'Average')
            plt.plot(x, Wrst, label = 'Worst')
            plt.legend()
            plt.title("Gráfico de Fitness ao longo das gerações\n* {} *".format(layout))
            plt.xlabel('Geração')
            plt.ylabel('Fitness')
            plt.show()

# Layout smallClassic

![image](Results/Genetic/Images/graphsmallClassic.png)

Pacman died! Score: -411  
Pacman emerges victorious! Score: 706  
Pacman died! Score: -183  
Pacman died! Score: -254  
Pacman emerges victorious! Score: 581  
Pacman died! Score: -323  
Pacman died! Score: -199  
Pacman died! Score: -395  
Pacman died! Score: -316  
Pacman died! Score: -146  
Average Score: -94.0  
Scores: -411.0, 706.0, -183.0, -254.0, 581.0, -323.0, -199.0, -395.0, -316.0, -146.0  
Win Rate: 2/10 (0.20)  
Record: Loss, Win, Loss, Loss, Win, Loss, Loss, Loss, Loss, Loss  

# Layout mediumClassic

![image](Results/Genetic/Images/graphmediumClassic.png)

Pacman died! Score: -33  
Pacman emerges victorious! Score: 799  
Pacman died! Score: 8  
Pacman died! Score: -186 
Pacman emerges victorious! Score: 976  
Pacman died! Score: -11  
Pacman died! Score: -278  
Pacman died! Score: -181  
Pacman died! Score: -287  
Pacman died! Score: 17  
Average Score: 82.4  
Scores: -33.0, 799.0, 8.0, -186.0, 976.0, -11.0, -278.0, -181.0, -287.0, 17.0  
Win Rate: 2/10 (0.20)  
Record: Loss, Win, Loss, Loss, Win, Loss, Loss, Loss, Loss, Loss  


# Layout originalClassic

![image](Results/Genetic/Images/graphoriginalClassic.png)

Pacman died! Score: -214  
Pacman died! Score: 768  
Pacman died! Score: 0  
Pacman died! Score: 697  
Pacman died! Score: 28  
Pacman died! Score: 208  
Pacman died! Score: 51  
Pacman died! Score: 825  
Pacman died! Score: -24  
Pacman died! Score: 3  
Average Score: 234.2  
Scores: -214.0, 768.0, 0.0, 697.0, 28.0, 208.0, 51.0, 825.0, -24.0, 3.0  
Win Rate: 0/10 (0.00)  
Record: Loss, Loss, Loss, Loss, Loss, Loss, Loss, Loss, Loss, Loss  

## Conclusões

É possível verificar o resultado esperado em todos os modelos, ao passar da gerações ocorreu o aumento do fitness, principalmente no início há um crescimento rápido, por fim establiza em um valor, porém ao se tratar de probabilidades tanto na criação de uma nova geração quanto na execução e escolha das ações do pacman, é comum surgir alguns melhores outros piores. Podemos ver que mesmo o melhor indíviduo ter ganhado um jogo no modelo original ao rodar ele 10 vezes ele não conseguiu repetir o sucesso, porém apresentando bons scores, uma grande dificuldade encontrada foi quando o pacman, para fugir dos fantasmas, é empurrado para longe de algumas poucas comidas que restaram, como sua exploração se baseia em probabilidade e existe muitos obstáculos, o modelo não apresentou bons resultados nesse estado do pacman muito longe de alguma comida, nos layouts menores essa dificuldade foi amenizada. Uma possibilidade de melhoria seria tentar remodelar para se basear menos em probabilidade em suas ações, considerando também o movimento estocástico dos fantasmas.

# Reinforcement Learning

Para o aprendizado por reforço foi escolhido o método Q-Learning para ser utilizado. Um método model-free e value-based, onde inicialmente iremos definir uma Q-Table ($Q[S, A]$) com conjunto de estados e ações e atualizando o valor de qualidade para cada $Q(s,a)$ ao decorrer do treinamento através de recompensa e a Q-function. A seguir será explicado a teoria utilizada e o modelo a ser implementado, após isso será apresentado o código e explicações das funções mais relevantes e por fim os gráficos gerados e conclusões do processo. Este segmento, assim como o modelo genético, não poderá ser executado, apesar de pré-gerados os códigos e resultados serão todos apresentados em markdown.

# Passos do Q-Learning

## Passo 1: Q-Table  

A tabela utilizada no Q-Learning é uma combinação de estados $s$ do conjunto $S$ e ações $a$ do conjunto $A$, sendo assim, cada $(s,a)$ possui um valor de qualidade dado por $Q(s,a)$.   
  
O ideal seria utilizar o estado completo do jogo, porém para poupar poder computacional, construímos os estados baseado em quais ações são permitidas na atual posição do pacman (Norte, Sul, Leste e Oeste), se possui algum fantasma perto (menor que 3.5 unidades de distância), posições relativas do fantasma e da comida mais próxima (Norte, Sul, Leste, Oeste, Nordeste, Sudeste, Noroeste e Sudoeste). O conjunto de ações consiste em Norte, Sul, Leste e Oeste.  
  
Sendo assim, obteremos uma tabela com a coluna sendo os estados e cada linha sendo as ações. O modelo irá analisar a cada nova posição do pacman qual estado ele está, se ainda não estiver na tabela ele é incluído e todos os elementos terão valor inicialmente 0.

## Passo 2: Escolhendo e perfomando a ação

A cada estado deve-se escolher uma ação para depois ser avaliada e o intuito é escolher a ação que maximizará o valor de Q para o estado atual, porém deve-se balancear o exploration e exploitation do agente, sendo assim o modelo possui um $\epsilon = 0.1$ que o fornece a probabilidade de 10% de escolher uma ação aleatória permitida naquele estado, evitando assim que sempre repita uma ação, mesmo que boa, pois há a possibilidade de uma melhor, ainda não explorada, existir.

## Passo 3: Medindo a recompensa

Para avaliar a ação e atualizar o valor em $Q(s,a)$ precisa-se definir uma recompensa para ação. A função de recompensa $R(s,a)$ para esse modelo se baseia no seguinte algoritmo:
* Se possuía um fantasma próximo:
    * Se a distância aumentou +30 pontos
    * Senão -30 pontos
* Senão:
    * Se comeu comida ou cápsula:
        * Se comeu comida +10 pontos
        * Se comeu cápsula +20 pontos
    * Senão:
        * Se aproximou da comida +15 pontos
        * Senão -15 pontos


## Passo 4: Atualizando a Q-Table

Cada elemento da Q-Table será atualizada através da equação de Bellman:  
  
$New Q(s,a) = Q(s,a) + \alpha(R(s,a) + \gamma maxQ(s,a) - Q(s,a))$  
  
Onde:  
$New Q(s,a)$ é o novo elemento em $Q(s,a)$.  
$Q(s,a)$ é o valor atual em Q-Table.  
$\alpha$ é o learning rate, nosso modelo será 0.2.  
$R(s,a)$ é a função de recompensa.  
$\gamma$ é o discount rate, nosso modelo será 0.8.  
$maxQ(s,a)$ é o maior Q-value associado a este estado.  

## Passo 5: Repetir os passos 2,3 e 4

O modelo deverá executar diversos episódios até aprender, idealmente poderíamos definir um critério de parada para o algoritmo, porém como iremos passar o número de treinos a ser realizado por linha de comando tivemos dificuldade em definir um critério de parada, sendo assim, para o layout de smallClassic será executado 10 mil episódios, para o mediumClassic 5 mil e o originalClassic 2mil. O número de treinos foi se reduzindo por limitações computacionais.

# Implementação do Q-learning

A implementação segue o seguinte algoritmo:

* No construtor:
    * Inicializa um dicionário para armazenar $Q[S,A]$
    * Inicializa os parâmetros utilizados na Q-Function
    * Inicializa atributos utilizados para avaliar a recompensa  
  
  
* Função getAction():
    * Analisa qual o estado atual
    * Avalia a recompensa
    * Atualiza Q-Table
    * Atualiza os atributos da classe
    * Escolhe uma nova ação
  

* Função final():
    * Analisa qual o estado atual
    * Avalia a recompensa
    * Atualiza Q-Table 
    * Reseta os atributos da classe
    * Finaliza o episódio


No arquivo Agents.py possui a implementação da classe QLearnAgent que será apresentada e explicada aqui. Segue abaixo o código completo e a explicação para as principais funções

Inicialmente temos o construtor da classe incializando a Q-Table com um dicionário e definindo outros atributos que serão utilizados.

```python
class QLearnAgent(Agent):

    # Inicializa o Agent com os atributos necessários
    def __init__(self, numTraining = 10):

        # Parâmetros do Q-Learning 
        self.alpha = 0.2
        self.epsilon = 0.1
        self.gamma = 0.8
        self.qValues = dict()

        # Acompanha resultados durante o episódio
        self.numTraining = int(numTraining)
        self.episodesSoFar = 0
        self.actionsSoFar = 0
        self.totalReward = 0
        
        # Usado para criar estado e recompensas
        self.lastState = None
        self.lastAction = None
        self.lastScore = None
        self.lastNumFood = None
        self.lastCaps = None
        self.lastDistGhost = None
        self.lastDistFood = None
        self.doNotEat = None
        self.ghostWasNear = False
        self.lastFoodPosition = None

        # Escreve em arquivos diferente dependendo do número de treino
        if(self.getNumTraining() < 2500):
            with open('episodesResults3.txt','w') as f:
                f.write("")
        elif(self.getNumTraining() < 5500):
            with open('episodesResults2.txt','w') as f:
                f.write("")
        else:
            with open('episodesResults1.txt','w') as f:
                f.write("")
```
A seguir são algumas funções para acessar, setar e incrementar os atributos.

```python

    # Funções para acessar os atributos

    def incrementEpisodesSoFar(self):
        self.episodesSoFar += 1

    def getEpisodesSoFar(self):
        return self.episodesSoFar
    
    def incrementAcionsSoFar(self):
        self.actionsSoFar += 1
    
    def getAcionsSoFar(self):
        return self.actionsSoFar

    def getNumTraining(self):
        return self.numTraining

    def setEpsilon(self, value):
        self.epsilon = value

    def getAlpha(self):
        return self.alpha

    def setAlpha(self, value):
        self.alpha = value

    def getGamma(self):
        return self.gamma
```

A função seguinte recebe o estado do jogo e os movimentos legais do pacman e retorna uma string descrevendo o estado do conjunto S

```python
    # Cria o estado baseado em direções legais, distância do fantasma mais próximo e sua posição relativa e da comida mais próxima
    def createState(self,state,legal):
        posG = np.array(state.getGhostPositions())
        posP = np.array(state.getPacmanPosition())
        posFood = state.getFood()
        numFood = state.getNumFood()

        minDistGhost = 99999
        indG = None
        for i in range(len(posG)):
            if(np.linalg.norm(posG[i]-posP) < minDistGhost):
                minDistGhost = np.linalg.norm(posG[i]-posP)
                indG =  i

        if(posG[indG][0] > posP[0]):
            if(posG[indG][1] > posP[1]):
                ghostRelativePosition = "upright"
            elif(posG[indG][1] == posP[1]):
                ghostRelativePosition = "right"
            else:
                ghostRelativePosition = "downright"

        elif(posG[indG][0] == posP[0]):
            if(posG[indG][1] > posP[1]):
                ghostRelativePosition = "up"
            else:
                ghostRelativePosition = "down"
        else:
            if(posG[indG][1] > posP[1]):
                ghostRelativePosition = "upleft"
            elif(posG[indG][1] == posP[1]):
                ghostRelativePosition = "left"
            else:
                ghostRelativePosition = "downleft"

        if(numFood > 0):
            minDistFood = 99999
            minFood = None
            for x in range(posFood.width):
                for y in range(posFood.height):
                    if(posFood[x][y]):
                        food = np.array([x,y])
                        distFood = np.linalg.norm(food-posP)
                        if(distFood<minDistFood):
                            minDistFood = distFood
                            minFood = food
            
            if(minFood[0] > posP[0]):
                if(minFood[1] > posP[1]):
                    foodRelativePosition = "upright"
                elif(minFood[1] == posP[1]):
                    foodRelativePosition = "right"
                else:
                    foodRelativePosition = "downright"

            elif(minFood[0] == posP[0]):
                if(minFood[1] > posP[1]):
                    foodRelativePosition = "up"
                else:
                    foodRelativePosition = "down"
            else:
                if(minFood[1] > posP[1]):
                    foodRelativePosition = "upleft"
                elif(minFood[1] == posP[1]):
                    foodRelativePosition = "left"
                else:
                    foodRelativePosition = "downleft"
            self.lastFoodPosition  = foodRelativePosition
        else:
            foodRelativePosition = self.lastFoodPosition

        if(minDistGhost < 3):
            ghostNear = True
        else:
            ghostNear = False
        
        # Retorna uma string descrevendo o estado
        return str(legal)+str(ghostNear)+str(ghostRelativePosition)+str(foodRelativePosition)
```
A função seguinte inicializa um novo Q(s,a)

```python

    # Caso o estado ainda não exista é gerado um para cada ação legal com valor Q = 0
    def initializeQValues(self, pacmanState, legal):
        self.qValues[pacmanState] = dict()
        for action in legal:
            if action not in self.qValues[pacmanState]:
                self.qValues[pacmanState][action] = 0.0
```
A seguinte função é utilizada para avaliar a recompensa da última ação e atualizar o valor na Q-Table
```python

    # Calcula a recompensa da última ação e atualiza  valor Q do estado e sua ação através da função do Q-learning
    def updateQValue(self, state,pacmanState, final_step=False):
        
        posG = np.array(state.getGhostPositions())
        posP = np.array(state.getPacmanPosition())
        posFood = state.getFood()

        numFoodEat = self.lastNumFood - state.getNumFood()
        numCapsEat = self.lastCaps - len(state.getCapsules())

        minDistGhost = 99999
        for i in range(len(posG)):
            if(np.linalg.norm(posG[i]-posP) < minDistGhost):
                minDistGhost = np.linalg.norm(posG[i]-posP)
        
        minDistFood = 99999
        for x in range(posFood.width):
            for y in range(posFood.height):
                if(posFood[x][y]):
                    food = np.array([x,y])
                    distFood = np.linalg.norm(food-posP)
                    if(distFood<minDistFood):
                        minDistFood = distFood

        if(self.ghostWasNear):
            if(minDistGhost >= self.lastDistGhost):
                didNotEat = 30
            else:
                didNotEat = -30
        else:
            if(numFoodEat == 0 or numCapsEat == 0):
                if(minDistFood > self.lastDistFood):
                    didNotEat = -15
                else:
                    didNotEat = 15
            else:
                didNotEat = 0

        reward = numFoodEat * 10 + numCapsEat * 20 + didNotEat
        self.totalReward += reward

        max_Q_value = 0.0
        if not final_step:
            max_Q_value = max(list(self.qValues[pacmanState].values()))
        self.qValues[self.lastState][self.lastAction] += (self.alpha * (reward + self.gamma * max_Q_value - self.qValues[self.lastState][self.lastAction]))
```
A função seguinte escolhe uma nova ação, com possibilidade de ser uma ação aleatória.

```python

    # Escolha a nova ação balanceando exploration e exploitation
    def epsilonGreedy(self, state,pacmanState, legal):
        if Directions.STOP in legal:
            legal.remove(Directions.STOP)

        if not self.ghostWasNear:
            if(self.lastAction!= None):
                if (Directions.REVERSE[self.lastAction] in legal) and len(legal)>1:
                    legal.remove(Directions.REVERSE[self.lastAction])

        probability = random.random()

        if probability < self.epsilon: # Escolhe ação aleatória
            random_action = random.choice(legal)
            return random_action

        max_Q_action = None # Escolhe a que já sabe ser melhor
        for action in legal:
            if max_Q_action == None:
                max_Q_action = action
            if self.qValues[pacmanState][action] > self.qValues[pacmanState][max_Q_action]:
                max_Q_action = action
        return max_Q_action
```
As duas funções a seguir são somente utilizadas para atualizar ou resetar os atributos da classe

```python

    # Atualiza atributos para um novo estado
    def updateAttributes(self, state,pacmanState, legal):

        posG = np.array(state.getGhostPositions())
        posP = np.array(state.getPacmanPosition())
        posFood = state.getFood()

        self.lastNumFood = state.getNumFood()
        self.lastCaps = len(state.getCapsules())

        minDistGhost = 99999
        for i in range(len(posG)):
            if(np.linalg.norm(posG[i]-posP) < minDistGhost):
                minDistGhost = np.linalg.norm(posG[i]-posP)
        
        minDistFood = 99999
        for x in range(posFood.width):
            for y in range(posFood.height):
                if(posFood[x][y]):
                    food = np.array([x,y])
                    distFood = np.linalg.norm(food-posP)
                    if(distFood<minDistFood):
                        minDistFood = distFood
        
        if(minDistGhost <= 3.5 ):
            self.ghostWasNear = True
        else:
            self.ghostWasNear = False
        self.lastDistGhost = minDistGhost
        self.lastDistFood = minDistFood
        self.lastState = self.createState(state,legal)
        self.lastAction = self.epsilonGreedy(state,pacmanState, legal)
        self.lastScore = state.getScore()

    # Reseta todos atributos (no fim do episódio)
    def resetAttributes(self):
        self.totalReward = 0
        self.actionsSoFar = 0
        self.lastState = None
        self.lastAction = None
        self.lastScore = None
```

A função getAction é chamada a cada passo do pacman e realiza os passos do Q-Learning utilizando as funções já implementadas

```python

    # Escolhe uma ação, inicializa o estado que o pacman está caso não exista ainda e move o pacman
    def getAction(self, state):

        legal = state.getLegalPacmanActions()

        if Directions.STOP in legal: # Remove a ação de parar
            legal.remove(Directions.STOP)
        
        if not self.ghostWasNear: # Se não possuir fantasmas próximos impede o pacman reverter
            if(self.lastAction!= None):
                if (Directions.REVERSE[self.lastAction] in legal) and len(legal)>1:
                    legal.remove(Directions.REVERSE[self.lastAction])
        
        pacmanState = self.createState(state,legal) # Analisa o estado atual

        if pacmanState not in self.qValues: # Insere o estado na Q-Table
            self.initializeQValues(pacmanState, legal)

        if self.lastState != None: # Avalia a recompensa e atualiza Q-Table
            self.updateQValue(state,pacmanState)

        self.updateAttributes(state,pacmanState, legal) # Atualiza os atributos da classe

        self.incrementAcionsSoFar()

        return self.lastAction
```
A função final é chamada após uma derrota ou vitória encerrando o episódio

```python

    # Chamada no fim do episódio e atualiza o último estágio
    def final(self, state):
        legal = state.getLegalPacmanActions()

        if Directions.STOP in legal:
            legal.remove(Directions.STOP)
        
        if not self.ghostWasNear:
            if(self.lastAction!= None):
                if (Directions.REVERSE[self.lastAction] in legal) and len(legal)>1:
                    legal.remove(Directions.REVERSE[self.lastAction])

        if self.lastState != None:
            self.updateQValue(state,self.createState(state,legal), final_step=True)


        if(self.getNumTraining() < 2500):
            with open('episodesResults3.txt','a') as f:
                f.write(str(self.totalReward)+" "+str(self.getAcionsSoFar())+" "+str(state.getScore())+"\n")
        elif(self.getNumTraining() < 5500):
            with open('episodesResults2.txt','a') as f:
                f.write(str(self.totalReward)+" "+str(self.getAcionsSoFar())+" "+str(state.getScore())+"\n")
        else:
            with open('episodesResults1.txt','a') as f:
                f.write(str(self.totalReward)+" "+str(self.getAcionsSoFar())+" "+str(state.getScore())+"\n")

        self.resetAttributes()

        self.incrementEpisodesSoFar()

        if self.getEpisodesSoFar() == self.getNumTraining():
            self.setAlpha(0)
            self.setEpsilon(0)


```

# Execução do Q-Learning

Para executar os treinos foi utilizado o seguinte código num notebook, o tempo de execução foi 1.84 horas.  
Assim foi escrito em arquivos separados os resultados de cada episódio e também os resultados das 10 execuções dos melhores agentes.


``` python
saida1 = !python pacman.py -p QLearnAgent -x 10000 -n 10010 -l smallClassic 
with open('results1.txt','w') as f:
    for i in saida1:
        f.write(str(i)+'\n')

saida2 = !python pacman.py -p QLearnAgent -x 5000 -n 5010 -l mediumClassic 
with open('results2.txt','w') as f:
    for i in saida2:
        f.write(str(i)+'\n')

saida3 = !python pacman.py -p QLearnAgent -x 2000 -n 2010 -l originalClassic 
with open('results3.txt','w') as f:
    for i in saida3:
        f.write(str(i)+'\n')
```

A seguir segue o código excutado para gerar os gráficos

``` python
import matplotlib.pyplot as plt
from matplotlib import ticker

def createGraphs(file,title,YLabel,EpNum,ind,fileGenerated):
    with open(file) as f:
        lines = f.readlines()
        xy = [[int(line.split()[0]),int(line.split()[1]),float(line.split()[2])] for line in lines]

    epResults = []
    for i in range(len(xy[0])):  
        aux1 = []
        eps = []
        for j in range(len(xy)):
            eps.append(j+1)
            aux1.append(xy[j][i])
        epResults.append(aux1)

    fig = plt.figure(figsize=(60, 6))
    ax = fig.add_subplot()
    ax = plt.axes()
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1000))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(250))
    plt.xlim(0, EpNum)
    plt.scatter(eps,epResults[ind])
    plt.tick_params(axis='x', which='major', labelsize=30)
    plt.tick_params(axis='y', which='major', labelsize=30)
    plt.title(title,fontsize=30)
    plt.xlabel('Episodes',fontsize=30)
    plt.ylabel(YLabel,fontsize=30)
    plt.savefig(fileGenerated)

createGraphs("episodesResults1.txt","Reward x Episodes smallClassic","Total Reward",10000,0,"TRsmall.png")
createGraphs("episodesResults1.txt","Actions x Episodes smallClassic","Total Actions",10000,1,"TAsmall.png")
createGraphs("episodesResults1.txt","Score x Episodes smallClassic","Total Score",10000,2,"TSsmall.png")

createGraphs("episodesResults2.txt","Reward x Episodes mediumClassic","Total Reward",5000,0,"TRmedium.png")
createGraphs("episodesResults2.txt","Actions x Episodes mediumClassic","Total Actions",5000,1,"TAmedium.png")
createGraphs("episodesResults2.txt","Score x Episodes mediumClassic","Total Score",5000,2,"TSmedium.png")

createGraphs("episodesResults3.txt","Reward x Episodes originalClassic","Total Reward",2000,0,"TRoriginal.png")
createGraphs("episodesResults3.txt","Actions x Episodes originalClassic","Total Actions",2000,1,"TAoriginal.png")
createGraphs("episodesResults3.txt","Score x Episodes originalClassic","Total Score",2000,2,"TSoriginal.png")
```

# Resultados do Q-Learning

## Layout smallClassic

![image](Results/Reinforcement/Images/TRsmall.png)
![image](Results/Reinforcement/Images/TAsmall.png)
![image](Results/Reinforcement/Images/TSsmall.png)

Pacman emerges victorious! Score: 921  
Pacman died! Score: -347  
Pacman emerges victorious! Score: 1137  
Pacman emerges victorious! Score: 893  
Pacman died! Score: -214  
Pacman emerges victorious! Score: 967  
Pacman died! Score: -109  
Pacman emerges victorious! Score: 938  
Pacman emerges victorious! Score: 931  
Pacman emerges victorious! Score: 884  
Average Score: 600.1  
Scores: 921.0, -347.0, 1137.0, 893.0, -214.0, 967.0, -109.0, 938.0, 931.0, 884.0  
Win Rate: 7/10 (0.70)  
Record: Win, Loss, Win, Win, Loss, Win, Loss, Win, Win, Win  


## Layout mediumClassic

![image](Results/Reinforcement/Images/TRmedium.png)
![image](Results/Reinforcement/Images/TAmedium.png)
![image](Results/Reinforcement/Images/TSmedium.png)

Pacman emerges victorious! Score: 1257  
Pacman emerges victorious! Score: 1320  
Pacman emerges victorious! Score: 1272  
Pacman emerges victorious! Score: 1462  
Pacman died! Score: 171  
Pacman emerges victorious! Score: 1247  
Pacman emerges victorious! Score: 1270  
Pacman emerges victorious! Score: 1317  
Pacman died! Score: -126  
Pacman died! Score: 72  
Average Score: 926.2  
Scores: 1257.0, 1320.0, 1272.0, 1462.0, 171.0, 1247.0, 1270.0, 1317.0, -126.0, 72.0  
Win Rate: 7/10 (0.70)  
Record:  Win, Win, Win, Win, Loss, Win, Win, Win, Loss, Loss 

## Layout originalClassic

![image](Results/Reinforcement/Images/TRoriginal.png)
![image](Results/Reinforcement/Images/TAoriginal.png)
![image](Results/Reinforcement/Images/TSoriginal.png)

Pacman died! Score: 392  
Pacman emerges victorious! Score: 2297  
Pacman died! Score: 615  
Pacman died! Score: 831  
Pacman died! Score: 976  
Pacman died! Score: 293  
Pacman died! Score: 762  
Pacman emerges victorious! Score: 2302  
Pacman died! Score: 885  
Pacman died! Score: 402  
Average Score: 975.5  
Scores: 392.0, 2297.0, 615.0, 831.0, 976.0, 293.0, 762.0, 2302.0, 885.0, 402.0  
Win Rate: 2/10 (0.20)  
Record: Loss, Win, Loss, Loss, Loss, Loss, Loss, Win, Loss, Loss 

## Conclusões

A visualização dos gráficos não ficou tão clara quanto do método genético, porém é possível evidenciar as seguintes observações. A recompensa total por episódio cresce com bons resultados logo no início, como não utilizamos um critério de parada para utilizar o numTraining possivelmente o metódo teria convergido antes do número total de execuções utilizado. Outra observação relevante é verificar que, apesar de ter episódios com um grande número de ações, não encontra-se um bom reward, muitas vezes por o pacman ter entrado em algum loop realizando ações sem valor. Por fim vale ressaltar no gráfico de score, esxite uma lacuna separando valores na parte superior e inferior do gráfico, como o pacman recebe uma grande recompensa de score ao obter vitória e uma grande penalização de escore ao obter derrota, essa lacuna está separando episódios vitoriosos de derrotas, e é possível verificar que a concentração de pontos na zona superior aumenta ao passar dos episódios.  

A grande dificuldade que atingiu esse método, também o genético, porém nesse o resultado foi mais visível, foi o pacman estar em estados semelhantes ao apresentado abaixo, o nosso modelo não prevê como evitar essas situações impossíveis de sair. 

<img src="Results/Reinforcement/Images/ImpossibleScape.png" alt="img" width="200"/>


# Comparação de ambos metódos

## Resultados

* smallClassic
    * Reforço:
        * Average Score: 600.1
        * Win Rate: 7/10 (0.70)
    * Genético:
        * Average Score: -94.0 
        * Win Rate: 2/10 (0.20)
  
* mediumClassic
    * Reforço:
        * Average Score: 926.2  
        * Win Rate: 7/10 (0.70)
    * Genético:
        * Average Score: 82.4 
        * Win Rate: 2/10 (0.20)
  
* originalClassic
    * Reforço:
        * Average Score: 975.5   
        * Win Rate: 2/10 (0.20)
    * Genético:
        * Average Score: 234.2 
        * Win Rate: 0/10 (0.00)

## Conclusões finais 

Fica claro como os resultados apresentados pelo aprendizado por reforço utilizando Q-Learning foram superiores, muito se deve por a escolha de ação do pacman não ser probabilístico e esse método armazenar mais estados para escolher ações, porém esse aspecto presente no nosso modelo genético não é intrinsíco do mesmo, pode ser alterado para apresentar melhorias levando a outro ponto de diferenciação de ambos os métodos, aprendizado por reforço foi encontrado muitos métodos com uma implementação direta e fácil adaptação para o pacman, como o Q-Learning utilizado, que buscam soluções ótimas. Enquanto no método genético exigiu uma maior criatividade ao modelar o comportamento do pacman e parâmetros a ser utilizado, semelhante a definir estados no Q-Learning, porém no genético encontramos uma dificuldade maior, possivelmente disso gerou resultados inferiores. Com isso vale ressaltar que o genético foi o primeiro método a ser implementado e com ele aprendemos e entendemos melhor como utilizar o framework do pacman e foram surgindo mais idéias que contribuiram para a implementação do Q-Learning.

## Contribuições individuais

Heigon Soldera 217638 - Início do método genético, modelagem e implementação do Q-Learning   
Rafael Galib 204904 - Finalização do método genético, gerar gráficos

Muitas contribuições foram realizadas simultaneamente a partir de discussões e implementação conjunta através de chamadas online.